# Setup

In [ ]:
config = {
    'n_col_metadata': 50, # FH135 has 46 columns
    'n_images': 3e4, # FH135 has 33783 images
    'raw_to_tiff_ratio': 5,
    'raw_image_size': 4.4e-3,
    'tiff_image_size': 13e-3,
    'n_referenced_images': 2e4,
    'default_byte_size': 4,
    'units': ('GB', 1024**3),
    'disk_to_ram_ratio': 10, # Conservative numbers are higher
    'n_flights': 20, # Currently there are six flights
    'processing_speed': 5e-3, # In GB/s. This is probably a conservative estimate.
    'n_full_metadata_queries_per_week': 100, # Should be conservative again
    'n_full_image_queries_per_week': 5,
    'minimum_processing_time': 60., # Assume no query uses less than this time in seconds
}

## Requirements Estimation

### Images

In [ ]:
# Per flight reqs
n_tiff = config['n_images'] / (1. + config['raw_to_tiff_ratio'])
n_raw = config['n_images'] - n_tiff
images_volume = config['tiff_image_size'] * n_tiff + \
    config['raw_image_size'] * n_raw
images_volume

In [ ]:
total_image_volume = images_volume * config['n_flights']

In [ ]:
# Total metadata cpu usage assuming every query retrieves all the metadata
image_time_estimate = (
    (
        # Time to process all the metadata
        total_image_volume
        / config['processing_speed']
        + config['minimum_processing_time']
    )
    * config['n_full_image_queries_per_week']
    / 3600. # Convert seconds to hours
)

### Metadata

In [ ]:
# Per flight reqs
table_size = (
    config['n_col_metadata'] * config['n_referenced_images'] 
    * config['default_byte_size'] / config['units'][1]
)
table_size

In [ ]:
total_metadata_volume = table_size * config['n_flights']
metadata_ram = total_metadata_volume * config['disk_to_ram_ratio']

In [ ]:
# Total metadata cpu usage assuming every query retrieves all the metadata
metadata_time_estimate = (
    (
        # Time to process all the metadata
        total_metadata_volume
        / config['processing_speed']
        + config['minimum_processing_time']
    )
    * config['n_full_metadata_queries_per_week']
    / 3600. # Convert seconds to hours
)

### Summarize Requirements

In [ ]:
print(
f'''Required volume for images: {total_image_volume:.2g} {config['units'][0]}
Images usage estimate: {image_time_estimate:.3g} hrs/week

Required volume for metadata: {total_metadata_volume:.2g} {config['units'][0]}
Required RAM for metadata: {metadata_ram:.2g} {config['units'][0]}
Metadata usage estimate: {metadata_time_estimate:.2g} hrs/week
'''
)

## Solutions